In [ ]:
import numpy as np
from PIL import Image

# ---------- load ----------
def load_gray(x):
    if isinstance(x, Image.Image):
        im = x.convert("L")
    else:
        im = Image.open(x).convert("L")
    return np.asarray(im, dtype=np.float32) / 255.0

# ---------- binarize with auto threshold & auto invert ----------
def otsu_threshold(arr01):
    """Otsu threshold on [0,1] image -> threshold in [0,1]"""
    # to uint8
    img = (arr01 * 255).astype(np.uint8)
    # histogram
    hist = np.bincount(img.ravel(), minlength=256).astype(np.float64)
    total = img.size
    sum_total = np.dot(np.arange(256), hist)

    sumB = 0.0
    wB = 0.0
    max_var = -1.0
    thresh = 127

    for t in range(256):
        wB += hist[t]
        if wB == 0:
            continue
        wF = total - wB
        if wF == 0:
            break
        sumB += t * hist[t]
        mB = sumB / wB
        mF = (sum_total - sumB) / wF
        var_between = wB * wF * (mB - mF) ** 2
        if var_between > max_var:
            max_var = var_between
            thresh = t
    return thresh / 255.0

def binarize_auto(arr01, thr=None, auto_invert=True, name=""):
    """
    thr=None이면 Otsu 자동 threshold.
    auto_invert=True이면 패턴 픽셀 비율이 너무 작거나(0에 가까움)/너무 크면(1에 가까움)
    invert 시도해서 더 그럴듯한 쪽 선택.
    """
    if thr is None:
        thr = otsu_threshold(arr01)

    b = (arr01 >= thr).astype(np.uint8)

    if not auto_invert:
        return b, thr, False

    # 패턴 비율
    frac = b.mean()  # 1의 비율
    # 너무 극단이면 invert 후보도 만들어 비교
    b_inv = 1 - b
    frac_inv = b_inv.mean()

    # "더 그럴듯" 기준: (1) 극단 피하기 (0.02~0.98 사이 선호)
    def score(f):
        # 0.5에 가까울수록 높은 점수, 극단이면 낮음
        return -abs(f - 0.5)

    use_inv = False
    # b가 너무 극단이면 invert 고려
    if frac < 0.01 or frac > 0.99:
        use_inv = score(frac_inv) > score(frac)
    else:
        # 극단이 아니면 원본 유지
        use_inv = False

    if use_inv:
        return b_inv, thr, True
    return b, thr, False

# ---------- metrics ----------
def ensure_same_shape(a, b):
    if a.shape != b.shape:
        raise ValueError(f"Shape mismatch: {a.shape} vs {b.shape}. 4개 이미지 크기 동일 필요.")
    return a, b

def edge(mask_bool):
    m = mask_bool.astype(np.uint8)
    gx = np.abs(np.diff(m, axis=1, prepend=m[:, :1]))
    gy = np.abs(np.diff(m, axis=0, prepend=m[:1, :]))
    return (gx + gy) > 0

def compute_metrics(ref_bin, pred_bin):
    ref_bin, pred_bin = ensure_same_shape(ref_bin, pred_bin)
    ref = ref_bin.astype(bool)
    pred = pred_bin.astype(bool)

    tp = np.logical_and(ref, pred).sum()
    fp = np.logical_and(~ref, pred).sum()
    fn = np.logical_and(ref, ~pred).sum()

    ref_area = int(ref.sum())
    pred_area = int(pred.sum())

    iou = tp / (tp + fp + fn + 1e-12)
    iou_err = 1.0 - iou

    # Area Error는 ref_area=0일 때 의미 없으니 NaN
    if ref_area == 0:
        area_err = float("nan")
        fp_rate = float("nan")
        fn_rate = float("nan")
    else:
        area_err = abs(pred_area - ref_area) / ref_area
        fp_rate = fp / ref_area
        fn_rate = fn / ref_area

    # Boundary F1 (ref edge가 없으면 의미 없음)
    re = edge(ref)
    pe = edge(pred)
    b_tp = np.logical_and(re, pe).sum()
    b_fp = np.logical_and(~re, pe).sum()
    b_fn = np.logical_and(re, ~pe).sum()

    b_prec = b_tp / (b_tp + b_fp + 1e-12)
    b_rec  = b_tp / (b_tp + b_fn + 1e-12)
    b_f1   = 2 * b_prec * b_rec / (b_prec + b_rec + 1e-12)

    return {
        "IoU": float(iou),
        "IoU Error": float(iou_err),
        "Area Error": float(area_err),
        "FP Rate": float(fp_rate),
        "FN Rate": float(fn_rate),
        "Boundary F1": float(b_f1),
        "Ref Area(px)": ref_area,
        "Pred Area(px)": pred_area,
    }

def pretty_print(title, m):
    def fmt(x):
        return "NaN" if (isinstance(x, float) and np.isnan(x)) else f"{x:.6f}"
    print(f"\n=== {title} ===")
    print(f"IoU            : {m['IoU']:.6f}")
    print(f"IoU Error      : {m['IoU Error']:.6f}")
    print(f"Area Error     : {fmt(m['Area Error'])}")
    print(f"FP Rate(over)  : {fmt(m['FP Rate'])}   (RefArea 기준)")
    print(f"FN Rate(under) : {fmt(m['FN Rate'])}   (RefArea 기준)")
    print(f"Boundary F1    : {m['Boundary F1']:.6f}")
    print(f"Ref Area(px)   : {m['Ref Area(px)']}")
    print(f"Pred Area(px)  : {m['Pred Area(px)']}")

# ---------- main ----------
def run_pipeline_metrics_auto(
    target, litho, resist, printed,
    thr_target=0.5,
    thr_litho=None,   # None이면 Otsu
    thr_resist=None,  # None이면 Otsu
    thr_printed=0.5,
    auto_invert=True,
    worst_by="IoU Error"
):
    t = load_gray(target)
    l = load_gray(litho)
    r = load_gray(resist)
    p = load_gray(printed)

    tb, _, _ = binarize_auto(t, thr_target, auto_invert=False)  # target은 보통 binary라 invert 안함
    lb, lthr, linv = binarize_auto(l, thr_litho, auto_invert=auto_invert, name="litho")
    rb, rthr, rinv = binarize_auto(r, thr_resist, auto_invert=auto_invert, name="resist")
    pb, _, _ = binarize_auto(p, thr_printed, auto_invert=False)

    print(f"\n[INFO] Litho threshold={lthr:.4f}, inverted={linv}, area={lb.sum()}")
    print(f"[INFO] Resist threshold={rthr:.4f}, inverted={rinv}, area={rb.sum()}")

    m_tl = compute_metrics(tb, lb)
    m_lr = compute_metrics(lb, rb)
    m_rp = compute_metrics(rb, pb)
    m_tp = compute_metrics(tb, pb)

    pretty_print("Target → Litho", m_tl)
    pretty_print("Litho → Resist", m_lr)
    pretty_print("Resist → Printed", m_rp)
    pretty_print("TOTAL: Target → Printed", m_tp)

    if worst_by not in ["IoU Error", "Area Error"]:
        raise ValueError('worst_by는 "IoU Error" 또는 "Area Error"만 가능합니다.')

    stages = {"Target → Litho": m_tl, "Litho → Resist": m_lr, "Resist → Printed": m_rp}

    # NaN 제외하고 worst 뽑기
    def stage_value(name):
        v = stages[name][worst_by]
        return -1 if (isinstance(v, float) and np.isnan(v)) else v

    worst_name = max(stages, key=stage_value)
    print(f"\n🔥 Worst stage by [{worst_by}]: {worst_name} = {stages[worst_name][worst_by]}")

    return {"Target→Litho": m_tl, "Litho→Resist": m_lr, "Resist→Printed": m_rp, "Total(Target→Printed)": m_tp}

In [ ]:
run_pipeline_metrics_auto("target.png","litho.png","resist.png","printed.png", auto_invert=True)


[INFO] Litho threshold=0.0745, inverted=False, area=916578
[INFO] Resist threshold=0.4353, inverted=False, area=131306

=== Target → Litho ===
IoU            : 0.147468
IoU Error      : 0.852532
Area Error     : 5.781128
FP Rate(over)  : 5.781128   (RefArea 기준)
FN Rate(under) : 0.000000   (RefArea 기준)
Boundary F1    : 0.000000
Ref Area(px)   : 135166
Pred Area(px)  : 916578

=== Litho → Resist ===
IoU            : 0.143257
IoU Error      : 0.856743
Area Error     : 0.856743
FP Rate(over)  : 0.000000   (RefArea 기준)
FN Rate(under) : 0.856743   (RefArea 기준)
Boundary F1    : 0.000000
Ref Area(px)   : 916578
Pred Area(px)  : 131306

=== Resist → Printed ===
IoU            : 0.956559
IoU Error      : 0.043441
Area Error     : 0.043441
FP Rate(over)  : 0.000000   (RefArea 기준)
FN Rate(under) : 0.043441   (RefArea 기준)
Boundary F1    : 0.000000
Ref Area(px)   : 131306
Pred Area(px)  : 125602

=== TOTAL: Target → Printed ===
IoU            : 0.874020
IoU Error      : 0.125980
Area Error     : 0.

{'Target→Litho': {'IoU': 0.14746808236723988,
  'IoU Error': 0.8525319176327602,
  'Area Error': 5.781128390275661,
  'FP Rate': 5.781128390275661,
  'FN Rate': 0.0,
  'Boundary F1': 0.0,
  'Ref Area(px)': 135166,
  'Pred Area(px)': 916578},
 'Litho→Resist': {'IoU': 0.14325676592717695,
  'IoU Error': 0.856743234072823,
  'Area Error': 0.856743234072823,
  'FP Rate': 0.0,
  'FN Rate': 0.856743234072823,
  'Boundary F1': 0.0,
  'Ref Area(px)': 916578,
  'Pred Area(px)': 131306},
 'Resist→Printed': {'IoU': 0.9565594869998324,
  'IoU Error': 0.0434405130001676,
  'Area Error': 0.04344051300016755,
  'FP Rate': 0.0,
  'FN Rate': 0.04344051300016755,
  'Boundary F1': 0.0,
  'Ref Area(px)': 131306,
  'Pred Area(px)': 125602},
 'Total(Target→Printed)': {'IoU': 0.8740199354648615,
  'IoU Error': 0.12598006453513855,
  'Area Error': 0.07075743900093219,
  'FP Rate': 0.029467469629936523,
  'FN Rate': 0.1002249086308687,
  'Boundary F1': 0.0739136320558159,
  'Ref Area(px)': 135166,
  'Pred Area

In [ ]:
run_pipeline_metrics_auto("target_complicated.png","litho_complicated.png","resist_complicated.png","printed_complicated.png", auto_invert=True)


[INFO] Litho threshold=0.0941, inverted=False, area=979926
[INFO] Resist threshold=0.4196, inverted=False, area=256054

=== Target → Litho ===
IoU            : 0.265611
IoU Error      : 0.734389
Area Error     : 2.764906
FP Rate(over)  : 2.764906   (RefArea 기준)
FN Rate(under) : 0.000000   (RefArea 기준)
Boundary F1    : 0.000000
Ref Area(px)   : 260279
Pred Area(px)  : 979926

=== Litho → Resist ===
IoU            : 0.261299
IoU Error      : 0.738701
Area Error     : 0.738701
FP Rate(over)  : 0.000000   (RefArea 기준)
FN Rate(under) : 0.738701   (RefArea 기준)
Boundary F1    : 0.000000
Ref Area(px)   : 979926
Pred Area(px)  : 256054

=== Resist → Printed ===
IoU            : 0.932670
IoU Error      : 0.067330
Area Error     : 0.067330
FP Rate(over)  : 0.000000   (RefArea 기준)
FN Rate(under) : 0.067330   (RefArea 기준)
Boundary F1    : 0.000000
Ref Area(px)   : 256054
Pred Area(px)  : 238814

=== TOTAL: Target → Printed ===
IoU            : 0.860323
IoU Error      : 0.139677
Area Error     : 0.

{'Target→Litho': {'IoU': 0.2656108726577313,
  'IoU Error': 0.7343891273422687,
  'Area Error': 2.7649061199712617,
  'FP Rate': 2.7649061199712617,
  'FN Rate': 0.0,
  'Boundary F1': 0.0,
  'Ref Area(px)': 260279,
  'Pred Area(px)': 979926},
 'Litho→Resist': {'IoU': 0.26129932260191074,
  'IoU Error': 0.7387006773980893,
  'Area Error': 0.7387006773980892,
  'FP Rate': 0.0,
  'FN Rate': 0.7387006773980892,
  'Boundary F1': 0.0,
  'Ref Area(px)': 979926,
  'Pred Area(px)': 256054},
 'Resist→Printed': {'IoU': 0.9326704523264624,
  'IoU Error': 0.0673295476735376,
  'Area Error': 0.06732954767353762,
  'FP Rate': 0.0,
  'FN Rate': 0.06732954767353762,
  'Boundary F1': 0.0,
  'Ref Area(px)': 256054,
  'Pred Area(px)': 238814},
 'Total(Target→Printed)': {'IoU': 0.8603228680162366,
  'IoU Error': 0.13967713198376341,
  'Area Error': 0.08246919651604624,
  'FP Rate': 0.030751616534564832,
  'FN Rate': 0.11322081305061107,
  'Boundary F1': 0.09028196369918594,
  'Ref Area(px)': 260279,
  'Pre

In [1]:
import numpy as np
from PIL import Image

# ---------- load ----------
def load_gray01(path_or_pil):
    im = path_or_pil if isinstance(path_or_pil, Image.Image) else Image.open(path_or_pil)
    im = im.convert("L")
    arr = np.asarray(im, dtype=np.float32) / 255.0
    return arr

# ---------- otsu ----------
def otsu_threshold(arr01):
    img = (arr01 * 255).astype(np.uint8)
    hist = np.bincount(img.ravel(), minlength=256).astype(np.float64)
    total = img.size
    sum_total = np.dot(np.arange(256), hist)

    sumB, wB = 0.0, 0.0
    max_var, thresh = -1.0, 127
    for t in range(256):
        wB += hist[t]
        if wB == 0:
            continue
        wF = total - wB
        if wF == 0:
            break
        sumB += t * hist[t]
        mB = sumB / wB
        mF = (sum_total - sumB) / wF
        var_between = wB * wF * (mB - mF) ** 2
        if var_between > max_var:
            max_var = var_between
            thresh = t
    return thresh / 255.0

def binarize(arr01, thr=None, invert=False):
    if thr is None:
        thr = otsu_threshold(arr01)
    b = (arr01 >= thr).astype(np.uint8)
    if invert:
        b = 1 - b
    return b, thr

# ---------- simple edges (like your code) ----------
def edge_map(mask01):
    m = mask01.astype(np.uint8)
    gx = np.abs(np.diff(m, axis=1, prepend=m[:, :1]))
    gy = np.abs(np.diff(m, axis=0, prepend=m[:1, :]))
    return ((gx + gy) > 0).astype(np.uint8)

# ---------- connected components (8-neighborhood) ----------
def count_components(mask01):
    # BFS on 1-pixels; pure numpy+python (OK for 2048^2, but still a bit heavy)
    h, w = mask01.shape
    visited = np.zeros((h, w), dtype=bool)
    comps = 0
    ones = np.argwhere(mask01 == 1)

    # quick exit
    if ones.size == 0:
        return 0

    # use a stack BFS
    for (y, x) in ones:
        if visited[y, x]:
            continue
        comps += 1
        stack = [(y, x)]
        visited[y, x] = True
        while stack:
            cy, cx = stack.pop()
            for dy in (-1, 0, 1):
                for dx in (-1, 0, 1):
                    if dy == 0 and dx == 0:
                        continue
                    ny, nx = cy + dy, cx + dx
                    if 0 <= ny < h and 0 <= nx < w and (not visited[ny, nx]) and mask01[ny, nx] == 1:
                        visited[ny, nx] = True
                        stack.append((ny, nx))
    return comps

# ---------- entropy ----------
def entropy(arr01, bins=64):
    hist, _ = np.histogram(arr01.ravel(), bins=bins, range=(0.0, 1.0), density=False)
    p = hist.astype(np.float64)
    p = p / (p.sum() + 1e-12)
    p = p[p > 0]
    return float(-(p * np.log2(p)).sum())

# ---------- complexity metrics ----------
def complexity_metrics(img_path, invert=False, thr=None):
    g = load_gray01(img_path)

    # grayscale-based
    ent = entropy(g, bins=64)
    gx = np.abs(np.diff(g, axis=1, prepend=g[:, :1]))
    gy = np.abs(np.diff(g, axis=0, prepend=g[:1, :]))
    grad_mean = float((gx + gy).mean())  # "texture/edge" 느낌

    # binary-based
    b, used_thr = binarize(g, thr=thr, invert=invert)
    area_frac = float(b.mean())
    e = edge_map(b)
    edge_density = float(e.mean())       # 경계 픽셀 비율(많을수록 복잡)
    perimeter = int(e.sum())             # 경계 길이(픽셀 수로 근사)
    area = int(b.sum())
    compactness = float((perimeter**2) / (area + 1e-12))  # 커질수록 복잡/울퉁불퉁
    components = count_components(b)

    return {
        "thr": float(used_thr),
        "invert": bool(invert),
        "entropy(gray)": ent,
        "mean_grad(gray)": grad_mean,
        "area_frac(bin)": area_frac,
        "edge_density(bin)": edge_density,
        "perimeter_px(bin)": perimeter,
        "compactness(peri^2/area)": compactness,
        "components(bin)": int(components),
    }

def compare_two(imgA, imgB, invertA=False, invertB=False, thrA=None, thrB=None):
    a = complexity_metrics(imgA, invert=invertA, thr=thrA)
    b = complexity_metrics(imgB, invert=invertB, thr=thrB)

    keys = [
        "entropy(gray)", "mean_grad(gray)",
        "area_frac(bin)", "edge_density(bin)",
        "perimeter_px(bin)", "compactness(peri^2/area)",
        "components(bin)"
    ]

    print("\n=== A ===")
    print(f"thr={a['thr']:.4f}, invert={a['invert']}")
    for k in keys:
        print(f"{k:28s}: {a[k]}")

    print("\n=== B ===")
    print(f"thr={b['thr']:.4f}, invert={b['invert']}")
    for k in keys:
        print(f"{k:28s}: {b[k]}")

    # 상대 비교(>1이면 B가 더 큼)
    print("\n=== Ratio (B / A) ===")
    for k in keys:
        denom = a[k] if a[k] != 0 else 1e-12
        print(f"{k:28s}: {b[k] / denom}")

    return a, b

In [4]:
A, B = compare_two("target.png", "target_complicated.png", thrA=0.5, thrB=0.5)


=== A ===
thr=0.5000, invert=False
entropy(gray)               : 0.20543556781260264
mean_grad(gray)             : 0.0009169578552246094
area_frac(bin)              : 0.0322260856628418
edge_density(bin)           : 0.0009150505065917969
perimeter_px(bin)           : 3838
compactness(peri^2/area)    : 108.97891481585606
components(bin)             : 5

=== B ===
thr=0.5000, invert=False
entropy(gray)               : 0.3355504257082207
mean_grad(gray)             : 0.0018515586853027344
area_frac(bin)              : 0.062055349349975586
edge_density(bin)           : 0.001848459243774414
perimeter_px(bin)           : 7753
compactness(peri^2/area)    : 230.940679040568
components(bin)             : 9

=== Ratio (B / A) ===
entropy(gray)               : 1.6333609086344203
mean_grad(gray)             : 2.0192407696307852
area_frac(bin)              : 1.925624787298581
edge_density(bin)           : 2.0200625325690464
perimeter_px(bin)           : 2.0200625325690464
compactness(peri^2/area) 